In [8]:
!pip install import_ipynb

import import_ipynb
import datGauss_alt as datGauss
import cliffGauss_alt as cliffGauss 
import scanData_alt as scanData

In [7]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import rankdata
from matplotlib import cm

def test_model(model, datagen, points, errmx=None, new=0, Ngrid=50, doplot=False):
    """
    Evaluate model predictions against a CliffGauss data generator.
    Plots results if doplot=True.
    """
    if points.ndim == 1:
        points = points.reshape(1, -1)

    if points.shape[1] != datagen.d:
        raise ValueError("Dimension mismatch")

    # Predictions and errors
    preds = np.zeros(points.shape[0])
    targs = np.zeros(points.shape[0])
    err = np.zeros(points.shape[0])

    for i in range(points.shape[0]):
        preds[i] = model.predict(points[i].reshape(1, -1))[0]
        targs[i] = datagen.sample_data(points[i].reshape(1, -1))[0]
        err[i] = abs(targs[i] - preds[i])

    if errmx is None:
        errmx = np.max(err)

    print(f"max err = {np.max(err)}")
    print(f"mean err = {np.mean(err)}")
    rankcor = np.corrcoef(rankdata(targs), rankdata(preds))[0, 1]
    print(f"rank cor = {rankcor}")

    rtn = {
        'maxerr': np.max(err),
        'meanerr': np.mean(err),
        'rankcor': rankcor,
        'err': err,
        'targs': targs
    }

    if doplot:
        xl, yl = (0.033, 0.967), (0.033, 0.967)

        # Plot surface using the new scanData function
        scanData.scan_data(datagen, Ngrid=Ngrid, docontour=True)

        # Overlay sample points
        for i in range(points.shape[0]):
            color = 'blue' if i < points.shape[0] - new else 'red'
            plt.scatter(points[i, 1], points[i, 0], s=err[i] * 100 / errmx, c=color)
            plt.plot(points[i, 1], points[i, 0], 'o', color=color)

        # For d>2, manually add contour of first two dims
        if datagen.d > 2:
            vec = [0.5] * (datagen.d - 2)
        else:
            vec = []

        xx = np.linspace(0, 1, Ngrid)
        yy = np.linspace(0, 1, Ngrid)
        res = np.zeros((Ngrid, Ngrid))
        for i in range(Ngrid):
            for j in range(Ngrid):
                input_point = np.array([xx[j], yy[i]] + vec).reshape(1, -1)
                res[i, j] = model.predict(input_point)[0]

        plt.contour(res, extent=[*xl, *yl], cmap=cm.cool, alpha=0.6)

    return rtn
